In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os
from util import read_json_from, write_json_to
# from data import pQTL_protein_df,pQTL_protein_ids,pQTL_protein_feature_matrix,pQTL_protein_idx_lookup 
from visualization_util import df_roc_analysis,df_precision_recall_analysis
from validation import validate_relations_in_df,full_report_calculation_by_size,simplifed_report_df,is_corum_verifiable
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
PATH_ROOT = os.getcwd().replace("\\","/").replace("/notebooks","")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
big_chungus_validated_cocomplex_pairs = big_chungus_validated[big_chungus_validated['cocomplex'] == 1]
big_chungus_validated_non_cocomplex_pairs = big_chungus_validated[big_chungus_validated['cocomplex'] == 0].sample(20000)

In [82]:
uniprot_to_idx_lookup = {p:idx for idx,p in enumerate(protein_concensus_ids)}

In [104]:
big_chungus_validated_cocomplex_pairs_new = pd.read_csv(f"{PATH_ROOT}/data_sources/Tissue/protein_consensus_validated_cocomplex_pairs.csv")

In [117]:
big_chungus_validated_cocomplex_pairs_filtered = big_chungus_validated_cocomplex_pairs_new[big_chungus_validated_cocomplex_pairs_new['coobserved'] >= 5]

In [118]:
big_chungus_validated_cocomplex_pairs_filtered.describe()

,pearson,coobserved,binary_interaction,cocomplex
count,16010.000000,16010.000000,16010.000000,16010.0
mean,0.245112,9.693004,0.570581,1.0
std,0.408291,2.467910,0.495009,0.0
min,-0.996751,5.000000,0.000000,1.0
25%,-0.038687,8.000000,0.000000,1.0
50%,0.270026,10.000000,1.000000,1.0
75%,0.565378,12.000000,1.000000,1.0
max,0.996346,13.000000,1.000000,1.0


In [22]:
from numpy import ma
A = [1, 2, 3, 4, 5, np.NaN]
B = [2, 3, 4, 5.25, np.NaN, 100]

print(ma.corrcoef(ma.masked_invalid(A), ma.masked_invalid(B)))

[[1.0 0.99838143945703]
 [0.99838143945703 1.0]]


In [31]:
A= [[1,2,3,4],
    [2,6,7,np.NaN],
    [2,np.NaN,99,9],
   [2,np.NaN,99,9]]
ma.corrcoef(ma.masked_invalid(A))

masked_array(
  data=[[1.0, 0.944911182523068, 0.2521206817557957, 0.2521206817557957],
        [0.944911182523068, 1.0, 0.9429553264604812, 0.9429553264604812],
        [0.2521206817557957, 0.9429553264604812, 1.0, 1.0],
        [0.2521206817557957, 0.9429553264604812, 1.0, 1.0]],
  mask=[[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],
  fill_value=1e+20)